# Sylhet Floods 2022
## post_process_preds.ipynb
This script processes the predictions created through inference of the NASA IMpact S1 ML model. We mosaic together the tiled outputs.

In [1]:
from pathlib import Path
import os
import sys
import pandas as pd
import numpy as np
import importlib
import shutil
import matplotlib.pyplot as plt
import rasterio as rio
from rasterio import warp
import zipfile
import geopandas as gpd
from osgeo import gdal
import json
from py_linq import Enumerable
from shapely.geometry import box
from shapely.geometry import Polygon
from osgeo import gdal
import fiona
from osgeo import gdal, ogr
import rioxarray

In [4]:
# Set the root path
rootPath = Path('Z:/media/mule/Projects/NASA/NIP/Data/Raster/')

In [2]:
# Set the module path for helpers scripts
module_path = os.path.abspath(os.path.join('C:/Users/alexa/Documents/GitHub/Sylhet2022Floods/scr/'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
# Import module from prep_raster containing helpful functions to use
import importlib
import helpers.prepare_flood_raster as prep_raster
importlib.reload(prep_raster)

<module 'Helpers.prepare_flood_raster' from 'C:\\Users\\alexsaunders\\Documents\\01_uoa\\04_git\\NIP\\Sylhet\\Helpers\\prepare_flood_raster.py'>

## PART 1: Mosaic the 256x256 10m resolution tiles together

In [11]:
# Define the input path and get the date folders
PredsPath = rootPath/'SylhetNASAImpact/5_Preds/'
PredsDatePaths = [item for item in list(PredsPath.iterdir()) if '2022' in str(item)]
dates = [str(item.stem) for item in PredsDatePaths]

In [12]:
PredsDatePaths

[WindowsPath('Z:/media/mule/Projects/NASA/NIP/Data/Raster/SylhetNASAImpact/5_Preds/20220501'),
 WindowsPath('Z:/media/mule/Projects/NASA/NIP/Data/Raster/SylhetNASAImpact/5_Preds/20220511'),
 WindowsPath('Z:/media/mule/Projects/NASA/NIP/Data/Raster/SylhetNASAImpact/5_Preds/20220513'),
 WindowsPath('Z:/media/mule/Projects/NASA/NIP/Data/Raster/SylhetNASAImpact/5_Preds/20220523'),
 WindowsPath('Z:/media/mule/Projects/NASA/NIP/Data/Raster/SylhetNASAImpact/5_Preds/20220525'),
 WindowsPath('Z:/media/mule/Projects/NASA/NIP/Data/Raster/SylhetNASAImpact/5_Preds/20220604'),
 WindowsPath('Z:/media/mule/Projects/NASA/NIP/Data/Raster/SylhetNASAImpact/5_Preds/20220606'),
 WindowsPath('Z:/media/mule/Projects/NASA/NIP/Data/Raster/SylhetNASAImpact/5_Preds/20220616'),
 WindowsPath('Z:/media/mule/Projects/NASA/NIP/Data/Raster/SylhetNASAImpact/5_Preds/20220618'),
 WindowsPath('Z:/media/mule/Projects/NASA/NIP/Data/Raster/SylhetNASAImpact/5_Preds/20220628'),
 WindowsPath('Z:/media/mule/Projects/NASA/NIP/Data

In [13]:
dates

['20220501',
 '20220511',
 '20220513',
 '20220523',
 '20220525',
 '20220604',
 '20220606',
 '20220616',
 '20220618',
 '20220628',
 '20220630',
 '20220712',
 '20220722',
 '20220724',
 '20220803',
 '20220805',
 '20220815',
 '20220827',
 '20220829']

In [15]:
raster=None

In [16]:
# Loop through all dates and stitch together the rasters
for i, date in enumerate(dates):
    
    # Define the output path for saving the mosaic
    MosaicPath = PredsPath/'Mosaic/v2' #PredsPath/(date+'/Mosaic')
    MosaicPath.mkdir(exist_ok=True)
    
    # Check if already exists in which case skip to the next 
    MosaicFile = MosaicPath/(str(date)+'.tif')
    if os.path.exists(MosaicFile):
        continue
    print(f"Creating file {MosaicFile}")
               
    # Get the tif files representing the surface water (note that GPM also provide flood water geotiffs which remove permanent water)
    PredsDatePath = PredsDatePaths[i]/'v2'
    PredsTiles = [str(item) for item in PredsDatePath.iterdir() if item.suffix =='.tif']

    # Mosaic together the surface water geotiffs on the same date
    print('Date: ', date, ', # tiles:', len(PredsTiles))

    # Run the function to create the mosaic 
    mosaic, output_meta = prep_raster.mosaic_rasters(PredsTiles)
    
    # Output the mosaic
    with rio.open(MosaicFile, 'w', **output_meta) as m:
        m.write(mosaic)
    print('Mosaic created and exported:', MosaicFile)

Creating file Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetNASAImpact\5_Preds\Mosaic\v2\20220501.tif
Date:  20220501 , # tiles: 876
Created list of tiles to mosaic
Mosaic created and exported: Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetNASAImpact\5_Preds\Mosaic\v2\20220501.tif
Creating file Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetNASAImpact\5_Preds\Mosaic\v2\20220511.tif
Date:  20220511 , # tiles: 900
Created list of tiles to mosaic
Mosaic created and exported: Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetNASAImpact\5_Preds\Mosaic\v2\20220511.tif
Creating file Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetNASAImpact\5_Preds\Mosaic\v2\20220513.tif
Date:  20220513 , # tiles: 900
Created list of tiles to mosaic
Mosaic created and exported: Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetNASAImpact\5_Preds\Mosaic\v2\20220513.tif
Creating file Z:\media\mule\Projects\NASA\NIP\Data\Raster\SylhetNASAImpact\5_Preds\Mosaic\v2\20220523.tif
Date:  20220523 , # tile